# Juliaで固有値問題を精度保証付き数値計算したい


## 固有値問題とは

固有値問題は与えられた行列の固有値および固有ベクトルを求めること。つまり、
$
\newcommand{\C}{\mathbb{C}}
\newcommand{\IC}{\mathbb{IC}}
\newcommand{\bX}{\bar{X}}
$

$$
Ax=\lambda x,\quad A\in \C^{n\times n}
$$

を満たす $\lambda\in \C$, $x\in \C^{n\times n}$ ($x\neq 0$) を求める問題を指す。

Juliaで固有値と固有ベクトルを求める際には、`LinearAlgebra`パッケージに含まれている関数`eigen`・`eigvals`・`eigvecs`で求めることができる。

|関数| 役割 |
|:-----------|:----------|
| eigen()    | 固有値・固有ベクトル両方  |
| eigvals()    | 固有値  |
| eigvecs()  | 固有ベクトル |

多分、Lapackの何かを呼び足してるはず。

In [11]:
using LinearAlgebra
val, vec  = eigen([1 2; 2 1])

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
2-element Array{Float64,1}:
 -1.0
  3.0
vectors:
2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

In [12]:
val#固有値

2-element Array{Float64,1}:
 -1.0
  3.0

In [13]:
vec#固有ベクトル

2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

In [14]:
eigvals([1 2; 2 1])#固有値

2-element Array{Float64,1}:
 -1.0
  3.0

In [15]:
eigvecs([1 2; 2 1])#固有ベクトル

2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

## 密行列の固有値問題に対する精度保証

密行列とは、行列内のほとんどの要素が $0$ でない行列のことを指す。ここでは、行列$A\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ の全ての固有値に対する精度保証アルゴリズムを考える。 

### 実装したいアルゴリズム
(1)
ある行列 $A$ の全ての近似固有ベクトル $\bX\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ を求める。

(2)
$A\cdot\bX$ の包含 $C\in\IC^{n\times n}$ を求める。

(3)
区間連立1次方程式 $\bX G=C$ の解集合の包含 $G\supset \bX^{-1}\cdot C$ ($G\in\IC^{n\times n}$) を求める。

(4)
$G\in\IC^{n\times n}$ に対してゲルシュゴリンの定理を適用することを考える。

#### (1) 行列 $A$ の全ての近似固有ベクトル $\bX\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ を求める。

行列 $A\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ が対角化可能であるとするとき、$\lambda_i$ に対する固有ベクトル$x^{(i)}\in\C^{n}$ $(i=1,2,\cdots,n)$ を並べた行列

$$
X :=[x^{(1)},x^{(2)},\cdots,x^{(n)}]\in\C^{n\times n}
$$

によって、$A$ は

$$
X^{-1}AX = D = \rm{diag}(\lambda_1,\lambda_2,\cdots,\lambda_n)
$$

と対角化される。

In [1]:
using LinearAlgebra

n =100;
A = randn(n,n);

λ, X = eigen(A);

In [57]:
λ #固有値

100-element Array{Complex{Float64},1}:
  -9.978167802123481 + 0.0im
  -9.381410794857814 + 0.0im
   -8.13197323343574 - 1.4599032020954632im
   -8.13197323343574 + 1.4599032020954632im
  -7.842945302102529 - 3.7818914935497054im
  -7.842945302102529 + 3.7818914935497054im
 -7.3788069663725295 - 4.375756128549122im
 -7.3788069663725295 + 4.375756128549122im
  -6.829171365183653 + 0.0im
  -6.724904090155478 - 1.8225633988152188im
  -6.724904090155478 + 1.8225633988152188im
  -6.476473550127473 - 6.797944455489788im
  -6.476473550127473 + 6.797944455489788im
                     ⋮
   5.943846778834983 + 0.3623580716041321im
   6.207025540249612 - 3.5853670145764394im
   6.207025540249612 + 3.5853670145764394im
   6.416757271516062 - 5.832232416517741im
   6.416757271516062 + 5.832232416517741im
   7.932581374289546 + 0.0im
    7.93549997313656 - 4.906255647203692im
    7.93549997313656 + 4.906255647203692im
   8.369974260881728 - 1.9555232971578593im
   8.369974260881728 + 1.9555232971578

In [18]:
X #固有ベクトル

100×100 Array{Complex{Float64},2}:
  -0.0471469-0.00428596im  …     0.0697578-0.0228959im
   0.0771909+0.0138135im         0.0324054+0.0540673im
  -0.0100356-0.0764188im        -0.0125529+0.0151425im
   0.0688002+0.0755691im           0.23993+0.0im
  -0.0930571+0.121296im         -0.0452431+0.0950594im
 -0.00383673-0.00805708im  …    -0.0399727-0.0528042im
   0.0670406-0.0626803im         0.0633569+0.052725im
    0.102931-0.102319im          -0.131716-0.0362948im
   0.0715991+0.0436799im         -0.150623-0.181584im
   0.0400742-0.0989377im        -0.0524599+0.0164176im
  -0.0138912-0.015144im    …    0.00494904+0.0210263im
   0.0738222+0.0672142im         0.0110069-0.162152im
  -0.0133222-0.0223845im        -0.0300042-0.019619im
            ⋮              ⋱  
  -0.0969147-0.0380148im         0.0190995-0.0868385im
   0.0902272-0.0398788im          0.035731-0.10258im
  -0.0392593-0.0528965im   …    -0.0602592-0.092994im
   -0.081594+0.0384713im         0.0414883+0.068602im
   -0.136309+

しかし、実際のところ上記の数値計算で得られる固有ベクトル $X$ は近似の値 $\bX\in \C^{n\times n}$ (以下、近似固有ベクトルという) である。しかし、近似固有ベクトルを並べた行列を使って、対角化したものについては、対角優位性が期待できる。この行列を $G$ とすると、

$$
G := \bX^{-1}A\bX
$$

となり、行列 $G$ の対角成分は行列 $A$ の固有値の近似となる。

また、対角優位性とは、ある行列$A=(a_{ij})$について、

$$
\left|a_{i i}\right|>\sum_{{j} \colon j \neq i}\left|a_{i j}\right| \quad(i=1,2, \ldots, n)
$$

であること。つまり、行列のどの対角成分の絶対値も、それと同じ列にある非対角成分の和よりも大きい時。

In [2]:
G = inv(X)*A*X; #G
norm(diag(G) - λ,Inf) # Gの対角成分は固有値λにほぼ一致する

7.287859939672374e-14

#### (2) $A\cdot\bX$ の包含 $C\in\IC^{n\times n}$ を求める。

``方法1``各演算を区間演算に変更した方式(参考：[interval_dot-mul.ipynb](https://www.risk.tsukuba.ac.jp/~takitoshi/tutorial/interval_dot-mul.html "タイトル"))

In [3]:
using IntervalArithmetic, BenchmarkTools

iA = map(Interval, A);
iX = map(Interval, X);

@btime C1 = $iA * $iX;

  77.030 ms (8 allocations: 312.91 KiB)


In [5]:
@show max(maximum(radius.(real(C1[:]))),maximum(radius.(imag(C1[:]))))

LoadError: UndefVarError: C1 not defined

``方法2``BLASを使う。int_num_linalg.jlの呼び出し。この方が早い。

In [4]:
using BenchmarkTools
include("IntervalLinearAlgebra.jl"); # int_mulを使用する
# @time C1 = mm_comp_interval_alt(iA,iX);
@time C = int_mul(A,X); # A, X: complex matrices (not interval)
@time C = int_mul(A,X); # A, X: complex matrices (not interval)
@time C = int_mul(A,X); # A, X: complex matrices (not interval)


  1.302181 seconds (4.65 M allocations: 222.687 MiB, 3.17% gc time)
  0.155475 seconds (84.04 k allocations: 4.628 MiB)
  0.163554 seconds (84.04 k allocations: 4.628 MiB)


In [23]:
max(maximum(radius.(real(C[:]))),maximum(radius.(imag(C[:]))))

4.574118861455645e-14

In [8]:
real(C)

100×100 Array{Interval{Float64},2}:
 [-0.957696, -0.957695]    …  [-0.261251, -0.26125]
  [0.583712, 0.583713]        [-0.890055, -0.890054]
 [-2.00467, -2.00466]          [0.974361, 0.974362]
 [-0.321647, -0.321646]       [-0.719869, -0.719868]
  [0.1279, 0.127901]          [-0.129917, -0.129916]
 [-0.972016, -0.972015]    …   [0.197544, 0.197545]
  [0.401453, 0.401454]         [1.02867, 1.02868]
  [1.07488, 1.07489]           [0.327513, 0.327514]
  [1.08726, 1.08727]           [1.38159, 1.3816]
  [1.32679, 1.3268]           [-0.533184, -0.533183]
 [-1.18749, -1.18748]      …  [-0.272931, -0.27293]
  [1.68066, 1.68067]           [0.938251, 0.938252]
  [0.351804, 0.351805]        [-0.0587759, -0.0587758]
   ⋮                       ⋱  
 [-0.517929, -0.517928]        [0.392765, 0.392766]
  [0.516897, 0.516898]        [-0.44515, -0.445149]
 [-0.732545, -0.732544]    …  [-0.248713, -0.248712]
 [-1.07352, -1.07351]         [-0.669768, -0.669767]
 [-0.712745, -0.712744]       [-0.771838, -0.

In [9]:
imag(C)

100×100 Array{Interval{Float64},2}:
 [-2.22508e-308, 2.22508e-308]  …   [0.327299, 0.3273]
 [-2.22508e-308, 2.22508e-308]      [0.437695, 0.437696]
 [-2.22508e-308, 2.22508e-308]      [0.130328, 0.130329]
 [-2.22508e-308, 2.22508e-308]     [-1.5532, -1.55319]
 [-2.22508e-308, 2.22508e-308]     [-0.058642, -0.0586419]
 [-2.22508e-308, 2.22508e-308]  …  [-0.160517, -0.160516]
 [-2.22508e-308, 2.22508e-308]      [0.568468, 0.568469]
 [-2.22508e-308, 2.22508e-308]     [-0.994355, -0.994354]
 [-2.22508e-308, 2.22508e-308]      [0.703024, 0.703025]
 [-2.22508e-308, 2.22508e-308]     [-0.844976, -0.844975]
 [-2.22508e-308, 2.22508e-308]  …  [-1.11569, -1.11568]
 [-2.22508e-308, 2.22508e-308]      [0.536674, 0.536675]
 [-2.22508e-308, 2.22508e-308]     [-0.308824, -0.308823]
   ⋮                            ⋱  
 [-2.22508e-308, 2.22508e-308]      [0.325833, 0.325834]
 [-2.22508e-308, 2.22508e-308]      [0.454863, 0.454864]
 [-2.22508e-308, 2.22508e-308]  …  [-0.425711, -0.42571]
 [-2.22508e-308

#### (3 )区間連立1次方程式 $\hat XG=C$ の解集合の包含 $G\supset\hat X^{-1}\cdot C$ を求める。

In [5]:
function verifylss_ib(A,ib) # verify the solution element-wisely
    b = mid.(real(ib)) + mid.(imag(ib))*im;
    x̄ = A\b;
    n = length(x̄);
    R = inv(A);
    #########
    #C_mid, C_rad = mm_ufp(R,A);
    G = Matrix{Float64}(I, n, n) - int_mul(R,A);
    α = opnorm(G,Inf)# Interval arithmetic
    #########
    if α < 1
        x̄ = map(Interval,x̄);
        r = A*x̄ - ib; # Interval arithmetic
        Rr = R*r;
        err = abs.(Rr) + supremum(norm(Rr,Inf))/(1-α)*(abs.(G)*ones(n)); # Interval arithmetic
    else
        println("Oh my way, verification is failed...")
        err = nan;
    end
    return x = (real(x̄) .± supremum.(err)) + im*(imag(x̄) .± supremum.(err))
end

verifylss_ib (generic function with 1 method)

In [24]:
G = similar(C);
for i = 1:n
    G[:,i] = verifylss_ib(X,C[:,i]);
end
real(G)

100×100 Array{Interval{Float64},2}:
 [-9.97817, -9.97816]         …  [-1.33891e-12, 1.32822e-12]
 [-1.42062e-12, 1.44033e-12]     [-1.24544e-12, 1.26573e-12]
 [-1.44355e-12, 1.4463e-12]      [-1.2612e-12, 1.26651e-12]
 [-1.44355e-12, 1.4463e-12]      [-1.25196e-12, 1.28567e-12]
 [-1.98669e-12, 1.97716e-12]     [-1.76054e-12, 1.74041e-12]
 [-1.98669e-12, 1.97716e-12]  …  [-1.76215e-12, 1.75355e-12]
 [-1.84089e-12, 1.85094e-12]     [-1.6172e-12, 1.64502e-12]
 [-1.84089e-12, 1.85094e-12]     [-1.61396e-12, 1.6711e-12]
 [-1.53564e-12, 1.52362e-12]     [-1.40897e-12, 1.34868e-12]
 [-2.01834e-12, 2.02317e-12]     [-1.78939e-12, 1.78731e-12]
 [-2.01834e-12, 2.02317e-12]  …  [-1.78803e-12, 1.80919e-12]
 [-2.16209e-12, 2.16544e-12]     [-1.8908e-12, 1.92338e-12]
 [-2.16209e-12, 2.16544e-12]     [-1.8834e-12, 1.94093e-12]
   ⋮                          ⋱  
 [-6.432e-12, 6.39268e-12]       [-5.55967e-12, 5.58426e-12]
 [-2.78455e-12, 2.77869e-12]     [-2.44422e-12, 2.45302e-12]
 [-2.78455e-12, 2.77

In [59]:
[real(diag(G)), λ]

2-element Array{Array{T,1} where T,1}:
 Interval{Float64}[[-9.97817, -9.97816], [-9.38142, -9.38141], [-8.13198, -8.13197], [-8.13198, -8.13197], [-7.84295, -7.84294], [-7.84295, -7.84294], [-7.37881, -7.3788], [-7.37881, -7.3788], [-6.82918, -6.82917], [-6.72491, -6.7249]  …  [6.20702, 6.20703], [6.41675, 6.41676], [6.41675, 6.41676], [7.93258, 7.93259], [7.93549, 7.9355], [7.93549, 7.9355], [8.36997, 8.36998], [8.36997, 8.36998], [9.34986, 9.34987], [9.72336, 9.72337]]
 Complex{Float64}[-9.978167802123481 + 0.0im, -9.381410794857814 + 0.0im, -8.13197323343574 - 1.4599032020954632im, -8.13197323343574 + 1.4599032020954632im, -7.842945302102529 - 3.7818914935497054im, -7.842945302102529 + 3.7818914935497054im, -7.3788069663725295 - 4.375756128549122im, -7.3788069663725295 + 4.375756128549122im, -6.829171365183653 + 0.0im, -6.724904090155478 - 1.8225633988152188im  …  6.207025540249612 + 3.5853670145764394im, 6.416757271516062 - 5.832232416517741im, 6.416757271516062 + 5.832232416517741

#### 4) $G\in\IC^{n\times n}$ に対してゲルシュゴリンの定理を適用することを考える。


## ゲルシュゴリンの包含定理
行列 $A=(A_{ij})\in \C^{n\times n}$ について、$A$ の全ての固有値 $\lambda_i$ は、$\bigcup_{1\le i \le n}  U_i$ の内部に存在する。

$$
A = \bigcup_{1\le i \le n}  U_i \mbox{（数式の意味がわからない）}
$$

ただし、

$$
U_i = \left\{z\in\mathbb{C}:|z-a_{ii}|\le \sum_{j\neq i}|a_{ij}|\right\}
$$

である。円盤領域 $U_i$ をゲルシュゴリン円板といい、$A$ が強い優対角性を持つとき、$A$ の対角成分が $A$ の固有値の良い近似となる。今回の場合、$G=(g_{ij})_{1\le i,j\le n}\in\IC^{n\times n}$ の対角成分 $(g_{ii})_{1\le i\le n}$ を $c\in \IC^{n}$ とし、各成分の半径を $\mathrm{rad}(c_i)$ とすると、

$$
\begin{array}{ll}
\mbox{ゲルシュゴリン円の中心:}&\mathrm{mid}(c_i)\\
\mbox{ゲルシュゴリン円の半径:}&r_i=\sum_{j\neq i}\rm{mag}(g_{ij})+\rm{rad}(c_{i})\\
\end{array}
$$

となる。

In [47]:
import IntervalArithmetic: mag, radius, mid
function mag(v::Complex{Interval{T}}) where T# mag function for complex interval vectors
    abs_v = abs(v);
    return max(abs_v.lo,abs_v.hi)
end

function radius(v::Complex{Interval{T}}) where T# mag function for complex interval vectors
    return sqrt(interval(radius(real(v)))^2 + interval(radius(imag(v)))^2)
end

function mid(v::Complex{Interval{T}}) where T# mag function for complex interval vectors
    return mid(real(v)) + mid(imag(v))*im
end

mid (generic function with 9 methods)

In [48]:
r = zeros(n);
c = diag(G);
# G_org = G;
# for i ∈ 1:n
#     G[i,i] = 0;
# end
for i = 1:n
    for j = 1:n
        if i != j
            r[i] += mag(G[i,j]);
        end
    end
    r[i] += mag(radius.(c[i]));
end
center = mid.(c);

In [60]:
iλ = (real(center) .± r) + im*(imag(center) .± r);
[real(iλ) imag(iλ)]

100×2 Array{Interval{Float64},2}:
 [-9.97817, -9.97816]  [-2.43792e-10, 2.43792e-10]
 [-9.38142, -9.38141]  [-2.30911e-10, 2.30911e-10]
 [-8.13198, -8.13197]  [-1.45991, -1.4599]
 [-8.13198, -8.13197]   [1.4599, 1.45991]
 [-7.84295, -7.84294]  [-3.7819, -3.78189]
 [-7.84295, -7.84294]   [3.78189, 3.7819]
 [-7.37881, -7.3788]   [-4.37576, -4.37575]
 [-7.37881, -7.3788]    [4.37575, 4.37576]
 [-6.82918, -6.82917]  [-2.48793e-10, 2.48793e-10]
 [-6.72491, -6.7249]   [-1.82257, -1.82256]
 [-6.72491, -6.7249]    [1.82256, 1.82257]
 [-6.47648, -6.47647]  [-6.79795, -6.79794]
 [-6.47648, -6.47647]   [6.79794, 6.79795]
   ⋮                   
  [5.94384, 5.94385]    [0.362358, 0.362359]
  [6.20702, 6.20703]   [-3.58537, -3.58536]
  [6.20702, 6.20703]    [3.58536, 3.58537]
  [6.41675, 6.41676]   [-5.83224, -5.83223]
  [6.41675, 6.41676]    [5.83223, 5.83224]
  [7.93258, 7.93259]   [-1.83806e-10, 1.83805e-10]
  [7.93549, 7.9355]    [-4.90626, -4.90625]
  [7.93549, 7.9355]     [4.90625, 4.90626]
 

In [63]:
sum(λ .∈ iλ)

100

In [ ]:
using Plots
plot!(index, max_rad, yscale=:log10,
    xlabel       ="Real",                #X軸のラベル
    ylabel       ="Imaginary",   #Y軸のラベル
    xlims        =(0,41),             #X軸の範囲
    title="gershgorin",       #タイトル
    linewidth    =2,                  #線幅
    seriestype = :scatter,            #点プロットに　
    size         =(400,300),          #プロットのサイズ
)

In [ ]:
for i in 1:n
    m = sum(mag.(imag(G[i,:])))-mag.(imag(diag(G)[i]))
    maggi =m +magg
end

In [ ]:
maggi = sum(mag.(imag(G[:])))-sum(mag.(imag(diag(G)[:])))

In [ ]:
magg = maggr + maggi

In [ ]:
radg = sum(radius.(real(diag(G)[:]))) + sum(radius.(imag(diag(G)[:])))

In [ ]:
mag.(real(diag(G)[1]))

In [ ]:
using IntervalArithmetic
function gershgorin(G)
    g=diag(G)
    center=[real(g),imag(g)]
    ##real
    for i =1:n
        re = real(g)
        radr[i]=radius(re[i])
    ##imag
     for j =1:n
        im = imag(g)
        radim[j]=radius(im[j])
end

対角優位
http://www.math.ritsumei.ac.jp/~yasutomi/jugyo/Numerical_Analysis/note5.pdf

計算途中がでる
http://www.cas.cmc.osaka-u.ac.jp/~paoon/misc/julia/post/cg/


http://infsup.jp/saito/materials/na19exer.pdf